# Graph isomorphism

Lets assume that we have two undirected graphs $G_1 = (V_1, E_1)$ and $G_2 = (V_2, E_2)$. They are isomorphic if we can find bijective mapping from $V_1$ to $V_2$ so that structure of mapped V1 is identical to V2. Obviously $n=|V_1|=|V_2|$.   

Approach is to form QUBO which elements are mappings of single vertex $(v_1 \in V_1, v_1 \in V_1)$. So there are altogether $n^2$ elements. We build following constraints: 
1. every vertex from G1 must be mapped to one vertex in G2 
2. every vertex from G2 must be mapped to one vertex in G1 
3. with mapping every e in E1 should be mapped to e in E2 
4. with mapping every e in E2 should be mapped to e in E1 (this is however not needed, 1 and 2 ensures bijection already) 

In [1]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector
import networkx as nx
from networkx.classes.function import path_weight
import random

In [2]:
def create_qubo(G1,G2,p):
    vertices=len(G1.vertices)
    Q = np.zeros(vertices*vertices, vertices*vertices)
    return Q